In [26]:
import numbers
import os
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.python.framework import ops, tensor_shape, tensor_util
from tensorflow.python.ops import math_ops, random_ops, array_ops
from tensorflow.python.layers import utils
from sklearn.cross_validation import train_test_split
import time
from sklearn.utils import shuffle

In [79]:
# load data
df = pd.read_csv("./data/model_test.csv", encoding='utf-8')
df = df.drop(['Unnamed: 0'], axis=1)
df.tail()

,home_score,away_score,result,ERA_x,WHIP_x,h/g_x,hr/g_x,so/g_x,bb/g_x,hbp/g_x,ERA_y,WHIP_y,h/g_y,hr/g_y,so/g_y,bb/g_y,hbp/g_y
473,5.0,4.0,1.0,5.47,1.70,2.214286,0.142857,1.500000,0.785714,0.357143,6.42,1.70,5.300000,0.433333,2.833333,1.633333,0.166667
474,11.0,9.0,1.0,5.83,1.58,2.571429,0.142857,1.642857,1.178571,0.214286,6.42,1.70,5.300000,0.433333,2.833333,1.633333,0.166667
475,16.0,3.0,1.0,4.35,1.35,7.214286,0.892857,3.035714,1.464286,0.464286,5.43,1.62,1.548387,0.290323,1.516129,1.225806,0.096774
476,7.0,6.0,1.0,8.71,2.01,6.200000,0.600000,3.500000,2.100000,0.200000,7.09,1.77,1.536585,0.121951,0.780488,0.487805,0.121951
477,10.0,3.0,1.0,10.80,1.92,2.600000,0.200000,0.600000,0.600000,0.000000,6.85,1.86,5.818182,0.590909,2.363636,2.181818,0.227273


In [93]:
df = df[df.result != 0]

In [114]:
# define X and y
X = df.ix[:, 3:]
y = pd.get_dummies(df.result)

In [115]:
for feature in X.columns:
    X[feature] = (X[feature] - X[feature].mean())/X[feature].std()
    
print("Averages")
print(X.mean())

print("\n Deviations")

print(pow(X.std(), 2))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Averages
ERA_x      9.489086e-18
WHIP_x     3.083953e-18
h/g_x     -1.233581e-17
hr/g_x     4.175198e-17
so/g_x     4.222643e-17
bb/g_x     4.791988e-17
hbp/g_x    2.324826e-17
ERA_y     -1.945263e-17
WHIP_y    -3.131398e-17
h/g_y      1.335589e-16
hr/g_y     7.472655e-17
so/g_y    -7.792912e-17
bb/g_y     1.511137e-16
hbp/g_y   -1.135132e-16
dtype: float64

 Deviations
ERA_x      1.0
WHIP_x     1.0
h/g_x      1.0
hr/g_x     1.0
so/g_x     1.0
bb/g_x     1.0
hbp/g_x    1.0
ERA_y      1.0
WHIP_y     1.0
h/g_y      1.0
hr/g_y     1.0
so/g_y     1.0
bb/g_y     1.0
hbp/g_y    1.0
dtype: float64


In [116]:
# Generate Traning and Validation Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Convert to np arrays so that we can use with Tensorflow
X_train = np.array(X_train).astype(np.float32)
X_test  = np.array(X_test).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
y_test  = np.array(y_test).astype(np.float32)

print(np.shape(X_train), np.shape(y_train))
print(np.shape(X_test), np.shape(y_test))

(327, 14) (327, 2)
(141, 14) (141, 2)


In [117]:
# Define parameters for the model
learning_rate = 0.01
batch_size = 33
n_epochs = 10000

# Create placeholders for features and labels
X = tf.placeholder(tf.float32, [None, 14], name='X_placeholder')
Y = tf.placeholder(tf.float32, [None, 2], name='Y_placeholder')

# Create weights and bias
w = tf.Variable(tf.random_normal(shape=[14, 2], stddev=0.01), name='weights')
b = tf.Variable(tf.zeros([1, 2], name='bias'))

# Build model that returns the logits
logits = tf.matmul(X, w) + b

# Define log loss function ==> cross entropy of softmax of logits
y = tf.nn.softmax(logits)
loss = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(y), reduction_indices=[1]))

In [88]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    n_batches = int(len(X_train)/batch_size)
    
    for i in range(n_epochs): # train the model
        
        # shuffle X, y
        X_train, y_train = shuffle(X_train, y_train)
        total_loss = 0
        
        for j in range(n_batches):
            X_batch, Y_batch = X_train[j*batch_size:(j+1)*batch_size], y_train[j*batch_size:(j+1)*batch_size]
            _, loss_batch = sess.run([optimizer, loss], feed_dict={X: X_batch, Y:Y_batch}) 
            total_loss += loss_batch
            
        if i % 1000 == 0:
            print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    print('Optimization Finished')


Average loss epoch 0: 0.7092207446694374
Average loss epoch 1000: 0.6675850674510002
Average loss epoch 2000: 0.6693468168377876
Average loss epoch 3000: 0.6708036884665489
Average loss epoch 4000: 0.6705411821603775
Average loss epoch 5000: 0.6707606837153435
Average loss epoch 6000: 0.6720704808831215
Average loss epoch 7000: 0.6639352217316628
Average loss epoch 8000: 0.6681230887770653
Average loss epoch 9000: 0.6676737368106842
Optimization Finished


In [121]:
training_size = X_train.shape[1]
test_size = X_test.shape[1]
num_features = 14
num_labels = 2


num_hidden = 7

graph = tf.Graph()
with graph.as_default():
    tf_train_set    = tf.constant(X_train)
    tf_train_labels = tf.constant(y_train)
    tf_valid_set    = tf.constant(X_test)
 
    
    print(tf_train_set)
    print(tf_train_labels)
    
    ## Note, since there is only 1 layer there are actually no hidden layers... but if there were
    ## there would be num_hidden
    weights_1 = tf.Variable(tf.truncated_normal([num_features, num_hidden]))
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    ## tf.zeros Automaticaly adjusts rows to input data batch size
    bias_1 = tf.Variable(tf.zeros([num_hidden]))
    bias_2 = tf.Variable(tf.zeros([num_labels]))
    
    
    logits_1 = tf.matmul(tf_train_set , weights_1 ) + bias_1
    rel_1 = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(rel_1, weights_2) + bias_2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_2, labels=tf_train_labels))
    optimizer = tf.train.GradientDescentOptimizer(.005).minimize(loss)
    
    
    ## Training prediction
    predict_train = tf.nn.softmax(logits_2)
    
    
    
    # Validation prediction
    logits_1_val = tf.matmul(tf_valid_set, weights_1) + bias_1
    rel_1_val    = tf.nn.softmax(logits_1_val)
    logits_2_val = tf.matmul(rel_1_val, weights_2) + bias_2
    predict_valid = tf.nn.softmax(logits_2_val)

Tensor("Const:0", shape=(327, 14), dtype=float32)
Tensor("Const_1:0", shape=(327, 2), dtype=float32)


In [122]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [123]:
num_steps = 100000
with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print(loss.eval())
    test_accuracy = []
    for step in range(num_steps):
        _,l, predictions = session.run([optimizer, loss, predict_train])
        test_accuracy.append(accuracy(predict_valid.eval(), y_test))
        
        if (step % 10000 == 0 or step == num_steps-1):
#               print(predictions[3:6])
              print('Loss at step %d: %f' % (step, l))
              print('Training accuracy: %.1f%%' % accuracy(predictions, y_train[:, :]))
              print('Testing accuracy: %.1f%%' % accuracy(predict_valid.eval(), y_test))

3.14003
Loss at step 0: 3.140029
Training accuracy: 52.0%
Testing accuracy: 57.4%
Loss at step 10000: 0.663013
Training accuracy: 60.6%
Testing accuracy: 56.7%
Loss at step 20000: 0.647391
Training accuracy: 64.5%
Testing accuracy: 56.7%
Loss at step 30000: 0.628001
Training accuracy: 66.1%
Testing accuracy: 57.4%
Loss at step 40000: 0.609186
Training accuracy: 68.8%
Testing accuracy: 58.2%
Loss at step 50000: 0.590150
Training accuracy: 67.9%
Testing accuracy: 59.6%
Loss at step 60000: 0.573551
Training accuracy: 68.5%
Testing accuracy: 59.6%
Loss at step 70000: 0.555376
Training accuracy: 71.3%
Testing accuracy: 58.9%
Loss at step 80000: 0.541738
Training accuracy: 72.5%
Testing accuracy: 55.3%
Loss at step 90000: 0.533926
Training accuracy: 72.5%
Testing accuracy: 56.0%
Loss at step 99999: 0.526665
Training accuracy: 72.5%
Testing accuracy: 61.0%


In [124]:
np.max(test_accuracy)

65.248226950354606